In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
## Install packages and import modules
!pip install -qU elasticsearch sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.8/412.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.9 MB/s eta 0:00:00


## Preprocessing

In [37]:
import os
import numpy as np
import pandas as pd
import json

In [42]:
dataset_path = "/content/drive/MyDrive/SSE/Dataset/data/data_jsonl/data.jsonl"
df = pd.read_json(path_or_buf=dataset_path, lines=True)
df.head()

,id,url,name,name_abbreviation,decision_date,docket_number,first_page,last_page,citations,volume,reporter,court,jurisdiction,cites_to,frontend_url,preview,casebody
0,1268383,https://api.capapi.org/v1/cases/1268383/,The State vs. John Owen,State v. Owen,1810-07,,260,262,"[{'type': 'official', 'cite': '2 Wheel. Cr. Ca...","{'volume_number': '2', 'barcode': '32044078486...",{'url': 'https://api.capapi.org/v1/reporters/1...,"{'name': 'Supreme Court of North Carolina', 'i...","{'name_long': 'North Carolina', 'name': 'N.C.'...",[],https://cite.capapi.org/wheel-cr-cas/2/260/,[],"{'status': 'ok', 'data': {'head_matter': 'SUPR..."
1,11956941,https://api.capapi.org/v1/cases/11956941/,DUNLOP et al. v. WEST,Dunlop v. West,1805,"Case No. 4,170",93,93,"[{'type': 'official', 'cite': '8 F. Cas. 93'},...","{'volume_number': '8', 'barcode': '32044046873...",{'url': 'https://api.capapi.org/v1/reporters/9...,{'name': 'United States Circuit Court for the ...,"{'name_long': 'North Carolina', 'name': 'N.C.'...",[{'cite': '2 Hayw. N. C. 346'}],https://cite.capapi.org/f-cas/8/93/11956941/,[],"{'status': 'ok', 'data': {'head_matter': 'Case..."
2,11645357,https://api.capapi.org/v1/cases/11645357/,The FORTUNA,The Fortuna,1815,"Case No. 4,954",494,500,"[{'type': 'official', 'cite': '9 F. Cas. 494'}...","{'volume_number': '9', 'barcode': '32044054565...",{'url': 'https://api.capapi.org/v1/reporters/9...,{'name': 'United States Circuit Court for the ...,"{'name_long': 'North Carolina', 'name': 'N.C.'...","[{'cite': '1 Brock. 299'}, {'cite': '3 Wheat. ...",https://cite.capapi.org/f-cas/9/494/,[],"{'status': 'ok', 'data': {'head_matter': 'Case..."
3,11272108,https://api.capapi.org/v1/cases/11272108/,D. K. FUTCH v. ATLANTIC COAST LINE RAILROAD CO...,Futch v. Atlantic Coast Line Railroad,1919-10-15,,282,284,"[{'type': 'official', 'cite': '178 N.C. 282'}]","{'volume_number': '178', 'barcode': '320440578...",{'url': 'https://api.capapi.org/v1/reporters/5...,"{'name': 'Supreme Court of North Carolina', 'i...","{'name_long': 'North Carolina', 'name': 'N.C.'...",[],https://cite.capapi.org/nc/178/282/,[],"{'status': 'ok', 'data': {'head_matter': 'D. K..."
4,11273534,https://api.capapi.org/v1/cases/11273534/,"MRS. SUSANNA WILLIAMS v. C. G. BAILEY, B. R. B...",Williams v. Bailey,1919-12-03,,630,633,"[{'type': 'official', 'cite': '178 N.C. 630'}]","{'volume_number': '178', 'barcode': '320440578...",{'url': 'https://api.capapi.org/v1/reporters/5...,"{'name': 'Supreme Court of North Carolina', 'i...","{'name_long': 'North Carolina', 'name': 'N.C.'...",[],https://cite.capapi.org/nc/178/630/,[],"{'status': 'ok', 'data': {'head_matter': 'MRS...."


In [43]:
columns = ['id', 'name', 'decision_date', 'jurisdiction', 'head_matter', 'text']
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97600 entries, 0 to 97599
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 97600 non-null  int64 
 1   url                97600 non-null  object
 2   name               97600 non-null  object
 3   name_abbreviation  97600 non-null  object
 4   decision_date      97600 non-null  object
 5   docket_number      97600 non-null  object
 6   first_page         97600 non-null  object
 7   last_page          97600 non-null  object
 8   citations          97600 non-null  object
 9   volume             97600 non-null  object
 10  reporter           97600 non-null  object
 11  court              97600 non-null  object
 12  jurisdiction       97600 non-null  object
 13  cites_to           97600 non-null  object
 14  frontend_url       97600 non-null  object
 15  preview            97600 non-null  object
 16  casebody           97600 non-null  objec

In [44]:
# reduce size of dataset for ease
data = df.sample(frac=0.2, random_state=42)
len(data)

19520

In [45]:
data['head_matter'] = data['casebody'].apply(lambda x: x['data']['head_matter'])
data['text'] = data['casebody'].apply(lambda x: x['data']['opinions'][0]['text'] if len(x['data']['opinions']) > 0 else None)

In [46]:
data = pd.DataFrame(data=data, columns=columns)
data.dropna(inplace=True)
data.head()

,id,name,decision_date,jurisdiction,head_matter,text
15626,8656636,E. C. ARMSTRONG v. J. M. KINSELL and NATIONAL ...,1913-12-10,"{'name_long': 'North Carolina', 'name': 'N.C.'...",E. C. ARMSTRONG v. J. M. KINSELL and NATIONAL ...,"Clark, C. J.\nTbe plaintiff executed to defend..."
43222,8615879,STATE v. ABRAHAM L. LUFF,1930-04-16,"{'name_long': 'North Carolina', 'name': 'N.C.'...","STATE v. ABRAHAM L. LUFF.\n(Filed 16 April, 19...","Brogden, J.\nThe defendant contended that the ..."
46310,8616722,STATE v. FRANK BALLARD,1944-03-22,"{'name_long': 'North Carolina', 'name': 'N.C.'...","STATE v. FRANK BALLARD.\n(Filed 22 March, 1944...",Pee Cueiam.\nAfter a careful consideration of ...
35755,8654035,"FIRST NATIONAL BANK OF SPARTANBURG, SOUTH CARO...",1924-10-22,"{'name_long': 'North Carolina', 'name': 'N.C.'...","FIRST NATIONAL BANK OF SPARTANBURG, SOUTH CARO...","Stacy, J.\nThe president and assistant cashier..."
40598,8554475,STATE OF NORTH CAROLINA v. HAROLD EMANUEL,1972-12-20,"{'name_long': 'North Carolina', 'name': 'N.C.'...",STATE OF NORTH CAROLINA v. HAROLD EMANUEL\nNo....,"GRAHAM, Judge.\nAfter the case was docketed an..."


In [49]:
## Preprocessing
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [41]:
# Function to clean and preprocess text
def clean_text(text):
    text = text.lower()

    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize text into words
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a sentence
    cleaned_text = ' '.join(words)

    return cleaned_text

In [ ]:
data['cleaned_text'] = data['text'].apply(clean_text)

In [51]:
data = data.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
data = data.drop(columns=['index'], axis=1)
data.head()


,id,name,decision_date,jurisdiction,head_matter,text
0,8656636,E. C. ARMSTRONG v. J. M. KINSELL and NATIONAL ...,1913-12-10,"{'name_long': 'North Carolina', 'name': 'N.C.'...",E. C. ARMSTRONG v. J. M. KINSELL and NATIONAL ...,"Clark, C. J.\nTbe plaintiff executed to defend..."
1,8615879,STATE v. ABRAHAM L. LUFF,1930-04-16,"{'name_long': 'North Carolina', 'name': 'N.C.'...","STATE v. ABRAHAM L. LUFF.\n(Filed 16 April, 19...","Brogden, J.\nThe defendant contended that the ..."
2,8616722,STATE v. FRANK BALLARD,1944-03-22,"{'name_long': 'North Carolina', 'name': 'N.C.'...","STATE v. FRANK BALLARD.\n(Filed 22 March, 1944...",Pee Cueiam.\nAfter a careful consideration of ...
3,8654035,"FIRST NATIONAL BANK OF SPARTANBURG, SOUTH CARO...",1924-10-22,"{'name_long': 'North Carolina', 'name': 'N.C.'...","FIRST NATIONAL BANK OF SPARTANBURG, SOUTH CARO...","Stacy, J.\nThe president and assistant cashier..."
4,8554475,STATE OF NORTH CAROLINA v. HAROLD EMANUEL,1972-12-20,"{'name_long': 'North Carolina', 'name': 'N.C.'...",STATE OF NORTH CAROLINA v. HAROLD EMANUEL\nNo....,"GRAHAM, Judge.\nAfter the case was docketed an..."


In [52]:
dict = data.to_dict(orient='records')
# with open("data_reduce_new.json", "w") as outfile:
#     json.dump(dict, outfile)

# docs = json.load(open("data_reduce_new.json"))

# Setup the Embedding Model

For this example, we're using `all-MiniLM-L6-v2`, part of the `sentence_transformers` library. You can read more about this model on [Huggingface](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-distilroberta-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), providing the cloud id and password in your deployment.

In [3]:
from elasticsearch import Elasticsearch

ELASTIC_CLOUD_ID = "SSE:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQwNTJlNmQ5YzgzM2I0NzIwOThmZDk3NmZmODg5MTQ2ZiQxZWI5OWIyMjEzMDc0ZDEyYTM0NGVlODQ4YzkzZDBhNQ=="
ELASTIC_API_KEY = "YWlrYXpJd0JuYnI4ZWpVYU9KM0Y6Q09pQVhhemdSUDZCYWFIangwYnA0QQ=="

# Create the client instance
client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    ca_certs= r"/content/drive/MyDrive/SSE/A2525B64D8BFD084D946539261844AC9A3F7DBDC.crt",
    verify_certs=True,
    timeout=60
)

<ipython-input-3-09b634853fa9>:7: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  client = Elasticsearch(


Confirm that the client has connected with this test.

In [4]:
print(client.info())

{'name': 'instance-0000000001', 'cluster_name': '052e6d9c833b472098fd976ff889146f', 'cluster_uuid': 'BZxW-B35RZ6cW7wplL9Uuw', 'version': {'number': '8.10.4', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'b4a62ac808e886ff032700c391f45f1408b2538c', 'build_date': '2023-10-11T22:04:35.506990650Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


<p>
  From previous preprocessing to clean data, directly fetch data from newly created json file save in drive.
</p>

## Index some test data

AS our client is set up and connected to our Elastic deployment.
Now we need some data to test out the basics of Elasticsearch queries.
<br> About Dataset: It is extracted from: <b>Caselaw Access Project: Hovard Law School </b>. <a href="https://case.law/bulk/download/">Link: North Carolina</a>.
<br>Preprocess clean data have folowing properties: <br>
- `id`
- `name`
- `decision_data`
- `jurisdiction`
- `head_matter`
- `text`
- `cleaned_text`

### Create an index

First ensure that you do not have a previously created index with the name `legal_index`.

In [5]:
client.indices.delete(index="legal_index_1", ignore_unavailable=True)

ObjectApiResponse({'acknowledged': True})

🔐 NOTE: at any time you can come back to this section and run the `delete` function above to remove your index and start from scratch.

Let's create an Elasticsearch index with the correct mappings for our test data.

In [26]:
# Define the mapping
mappings = {
    "properties": {
        "text_vector": {
            "type": "dense_vector",
            "dims": 768,
            "index": "true",
            "similarity": "cosine"
        }
    }
}

# Create the index
client.indices.create(index='legal_index_1', mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'legal_index_1'})

### Index test data

Run the following command to upload some test data, containing information about 10 popular programming books from this [dataset](https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json).
`model.encode` will encode the text into a vector on the fly, using the model we initialized earlier.

In [ ]:
import json
path = "/content/drive/MyDrive/SSE/data_reduce_new.json"
docs = json.load(open(path))
docs = docs[:1000] ## Only taking a section of dataset due to lack of computation power and time limit


In [ ]:
for d in docs:
    if 'jurisdiction' in d:
        del d['jurisdiction']
    else:
        print("Key Not Present")

    if 'cleaned_text' in d:
        del d['cleaned_text']
    else:
        print("Key Not Present")

In [ ]:
operations = []
for doc in docs:
    operations.append({"index": {"_index": "legal_index_1"}})
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)
client.bulk(index="legal_index_1", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 5442, 'items': [{'index': {'_index': 'legal_index_1', '_id': 'm4zCzowBQAMjNS00F6Ih', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'legal_index_1', '_id': 'nIzCzowBQAMjNS00F6Ih', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'legal_index_1', '_id': 'nYzCzowBQAMjNS00F6Ih', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'legal_index_1', '_id': 'nozCzowBQAMjNS00F6Ih', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, 

## Aside: Pretty printing Elasticsearch responses

Your API calls will return hard-to-read nested JSON.
We'll create a little function called `pretty_response` to return nice, human-readable outputs from our examples.

In [35]:
def pretty_response(response):
    if len(response['hits']['hits']) == 0:
        print('Your search returned no results.')
    else:
        for hit in response['hits']['hits']:
            id = hit['_id']
            id_ = hit['_source']['id']
            score = hit['_score']
            name = hit['_source']['name']
            decision_date = hit['_source']['decision_date']
            head_matter = hit["_source"]["head_matter"]

            pretty_output = (f"\nID: {id}\nindex: {id_}\nName: {name}\ndecision_date: {decision_date}\ndhead_matter: {head_matter}\nScore: {score}")
            print(pretty_output)


## Making queries

Now that we have indexed the books, we want to perform a semantic search for books that are similar to a given query.
We embed the query and perform a search.

In [28]:
demo_text = "Clark, C. J.\nTbe plaintiff executed to defendant Burusell two notes for $400 eacb, payable at tbe National Bank of New Bern, respectively,- on 3 January, 1914, and 3 September, 1914, for tbe balance due on purchase of a “merry-go-round” on wbicb be bad made a casb payment. These notes were deposited with tbe defendant bank for collection. On arrival of tbe machine, being dissatisfied with its condition, tbe plaintiff brought this action, alleging false representation and breach of warranty and asking damages to tbe extent of tbe balance, of tbe purchase money and an injunction against tbe defendant Kinsell from negotiating or transferring said notes and against tbe bank to prevent its parting with tbe custody thereof until tbe further orders of tbe court.\nTbe defendant Kinsell entered a special appearance and asked to dismiss tbe action and to vacate and dissolve tbe 'restraining order, upon tbe ground that there bad been no personal service of tbe summons upon him and no appearance, or acceptance of service, and asking that tbe injunction be dissolved and that tbe action be dismissed.\n. Under tbe system of procedure prior to tbe adoption of The Code, injunctions were special or common. Tbe former was where tbe injunction itself was tbe relief sought, while a common injunction was an ancillary proceeding; but under Tbe Code all injunctions are'simply ancillary proceedings and cannot issue except when there is an action pending in court, in wbicb jurisdiction has been obtained in one of tbe modes recognized by tbe statute. These are fully discussed and distinguished in Bernhardt v. Brown, 118 N. C., 701. They are:\n(1) Personal service, or, in lieu thereof, acceptance of service or a waiver by appearance.\n(2) Proceedings in rem, in wbicb tbe court already has jurisdiction of the, res as to enforce some lien or a partition of property in its control, and tbe like. In these cases publication of the' summons or notice may be made, but the judgment has no personal force, not even for the costs, being limited to acting upon the property. •\n(3) Proceedings quasi in rem, in which cases the court acquires jurisdiction by attaching .property of a nonresident or of an absconding debtor (Winfree v. Bagley, 102 N. C., 515), and in similar eases, and the judgment has no effect • beyond the enforcement of the judgment out of the property seized by the attachment. In such cases publication of the summons or notice may be made based upon the jurisdiction of the property attached. Revisal, 442 and 442 (3); Grocery Co. v. Bag Co., 142 N. C., 174.\nProceedings in. divorce are sui generis, as the' judgment tliereih merely declares a personal status, and publication of the summons is allowed without the acquisition of jurisdiction by attachment of property, where the défendant is a nonresident, the court having jurisdiction of the person of the plaintiff.\nThe distinction between the above proceedings or methods of bringing parties into court is fully pointed out in Bernhardt v. Brown, supra, p. 706, with citation of authorities: Pennoyer v. Neff, 95 U. S., 714; Winfree v. Bagley, 102 N. C., 515; Long v. Insurance Co., 114 N. C., 465; Heilbetter v. Oil Co., 112 U. S., 294. Bernhardt v. Brown has been repeatedly cited; see Anno. Ed.\nIn this case there was no personal service on the defendant Kinsell nor acceptance of service nor waiver thereof by an appearance. He entered a special appearance and asked to dissolve the injunction and dismiss the proceeding. This is not a proceeding in rem to enforce any lien upon the property or to make partition thereof. Nor has jurisdiction been acquired as in a proceeding quasi in rem, because there has been no attachment issued and levied. An injunction granted before the issuing of a summons is irregular and will be vacated on motion. McArthur v. McEachin, 64 N. C., 72; Hirsh v. Whitehead, 65 N. C., 516. For a stronger reason, it must be vacated when no summons has been served on Kinsell and jurisdiction has not been acquired either by attachment -or by the court being in control of the res.\nThe injunction, therefore, was properly dismissed- as to Ein-sell and also as to the bank,* because as to the latter no cause of action was stated in the absence of the defendant Einsell. We see no advantage to the plaintiff in an injunction against the bank nor even as against Einsell, which cannot be had by the attachment' when procured. Moreover, an injunction as to a nonresident is improvident, for it can have no effect — usually, at least — except in personam. Warlick v. Reynolds, 151 N. C., 606.\nJurisdiction can be acquired as to Einsell by the service of an attachment upon the notes (Revisal, 777) and the publication of a notice based on the jurisdiction thus acquired. Best v. Mortgage Co., 128 N. C., 351; Grocery Co. v. Bag Co., 142 N. C., 180. In Winfree v. Bagley, 102 N. C., 515, it is held in a well considered opinion by Shepherd, J., that “ ‘a chose in action is property, and embraced in the terms of The Code which provides for service by publication’ when the defendant is not a resident of the State, but has property therein.” That case has been repeatedly cited since. See Anno. Ed.\nIn this case there was no publication of notice nor acquire^ ment of jurisdiction by attachment of the notes. The plaintiff did not ask to amend his proceeding by making the attachment and publication, and the judgment below dismissing the action is\nAffirmed."

In [ ]:
response = client.search(
    index="legal_index_1",
    knn={
      "field": "text_vector",
      "query_vector": model.encode(demo_text),
      "k": 10,
      "num_candidates": 100
    }
)

In [36]:
pretty_response(response)


ID: m4zCzowBQAMjNS00F6Ih
index: 8656636
Name: E. C. ARMSTRONG v. J. M. KINSELL and NATIONAL BANK OF NEW BERN
decision_date: 1913-12-10
dhead_matter: E. C. ARMSTRONG v. J. M. KINSELL and NATIONAL BANK OF NEW BERN.
(Filed 10 December, 1913.)
1. Injunctions — Distinctions Abolished — Code Practice — Interpretation of Statutes.
Under our Code practice the difference between special and common injunctions has been abolished, and they are ancillary to the relief sought in the actioh, and dependent upon service of process upon the defendant therein in accordance with the modes recognized by statute.
2. Injunctions — Bills and Notes — Banks and Banking — Nonresident Defendant — Process—Attachment.
Where the maker of a note brings his action against a nonresident payee to impeach his note upon the ground of fraud or false representations in its procurement, and seeks an injunction restraining the payee from further negotiating it, and a resident bank, where it had been deposited, from parting 

## Making queries

Now that we have indexed the books, we want to perform a semantic search for books that are similar to a given query.
We embed the query and perform a search.

In [ ]:
response = client.search(
    index="book_index",
    knn={
      "field": "title_vector",
      "query_vector": model.encode("javascript books"),
      "k": 10,
      "num_candidates": 100
    }
)

pretty_response(response)